In [1]:
import os
import requests
from dotenv import load_dotenv
import json
from datetime import datetime

class TickTickAPI:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'accept': 'application/json, text/plain, */*',
            'accept-language': 'zh-CN,zh;q=0.9',
            'content-type': 'application/json',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.session.headers.update(self.headers)
    
    def login(self, username, password):
        """登录滴答清单"""
        login_url = "https://api.dida365.com/api/v2/user/signon?wc=true&remember=true"
        payload = {
            "username": username,
            "password": password
        }
        response = self.session.post(login_url, json=payload)
        print(f"Login response status: {response.status_code}")
        return response.ok
    
    def get_tasks(self, project_id=None):
        """获取任务列表"""
        if project_id:
            # 获取特定项目的任务
            url = f"https://api.dida365.com/api/v2/project/{project_id}/tasks"
            response = self.session.get(url)
            print(f"Project tasks response status: {response.status_code}")
            return response.json() if response.ok else []
        else:
            # 获取所有任务
            url = "https://api.dida365.com/api/v2/batch/check/0"
            response = self.session.get(url)
            print(f"All tasks response status: {response.status_code}")
            if response.ok:
                data = response.json()
                return data.get('syncTaskBean', []) if isinstance(data, dict) else []
            return []
    
    def get_projects(self):
        """获取所有项目/清单"""
        url = "https://api.dida365.com/api/v2/projects"
        response = self.session.get(url)
        print(f"Projects response status: {response.status_code}")
        return response.json() if response.ok else []

# 测试代码
def print_task_info(task):
    """打印任务信息"""
    print(f"\n标题: {task.get('title')}")
    if task.get('startDate'):
        print(f"开始时间: {task.get('startDate')}")
    if task.get('dueDate'):
        print(f"截止时间: {task.get('dueDate')}")
    print(f"完成状态: {'已完成' if task.get('completedTime') else '未完成'}")
    if task.get('tags'):
        print(f"标签: {', '.join(task.get('tags'))}")
    if task.get('content'):
        print(f"内容: {task.get('content')}")
    print("---")

# 主流程
load_dotenv()
username = os.getenv('滴答清单账号')
password = os.getenv('滴答清单密码')

api = TickTickAPI()
if api.login(username, password):
    print("\n登录成功！")
    
    # 获取项目列表
    print("\n获取项目列表...")
    projects = api.get_projects()
    if projects:
        print(f"获取到 {len(projects)} 个项目")
        print("\n项目列表：")
        for project in projects:
            print(f"项目名称: {project.get('name')}")
            print(f"项目ID: {project.get('id')}")
            
            # 获取该项目的任务
            print(f"\n获取项目 '{project['name']}' 的任务...")
            project_tasks = api.get_tasks(project['id'])
            
            if project_tasks:
                print(f"获取到 {len(project_tasks)} 个任务")
                if len(project_tasks) > 0:
                    print("\n任务示例：")
                    for task in project_tasks[:2]:  # 只显示前两个任务
                        print_task_info(task)
            print("\n" + "="*50)
        
        # 保存所有数据到文件
        data = {
            'projects': projects,
            'tasks_by_project': {
                project['name']: api.get_tasks(project['id'])
                for project in projects
            }
        }
        
        with open('ticktick_data.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print("\n所有数据已保存到 ticktick_data.json")
else:
    print("登录失败！")

Login response status: 200

登录成功！

获取项目列表...
Projects response status: 200
获取到 8 个项目

项目列表：
项目名称: 🌈将来也许
项目ID: 2fb44a8b86e06c26008a48ff

获取项目 '🌈将来也许' 的任务...
Project tasks response status: 200
获取到 13 个任务

任务示例：

标题: 构建学习体系 
完成状态: 未完成
标签: 学习, 年度目标
---

标题: 完成一次蹦极   （1026-1114）
完成状态: 未完成
标签: 生活, 年度目标
---

项目名称: 📌例行日程
项目ID: 00974172a59021ea5049e8cb

获取项目 '📌例行日程' 的任务...
Project tasks response status: 200
获取到 10 个任务

任务示例：

标题: 【健康】洗牙  
开始时间: 2026-07-24T11:00:00.000+0000
截止时间: 2026-07-24T11:00:00.000+0000
完成状态: 已完成
标签: 生活, 健康
---

标题: 健康无忧保单缴费
开始时间: 2026-05-22T16:00:00.000+0000
截止时间: 2026-05-22T16:00:00.000+0000
完成状态: 未完成
标签: 生活, 财务
---

项目名称: 👉工作清单
项目ID: a2fe4ba494a95daf3765309b

获取项目 '👉工作清单' 的任务...
Project tasks response status: 200
获取到 5 个任务

任务示例：

标题: 邮件需求确认
开始时间: 2025-02-18T03:45:00.000+0000
截止时间: 2025-02-18T03:45:00.000+0000
完成状态: 已完成
---

标题: ISC 展会设计确认
开始时间: 2025-02-18T01:00:00.000+0000
截止时间: 2025-02-18T02:00:00.000+0000
完成状态: 已完成
---

项目名称: ✨生活清单
项目ID: 7d1f4380834bdaa7446a1746

获取项目

In [26]:
"""第三步：使用授权码获取访问令牌"""
url = "https://dida365.com/oauth/token"

# 使用 Basic Auth
auth = ("ohRz0P8Kc696Szc5WQ", "FJbG)i5T1L+n3w(Xrpw%2la^#l1$(vI8")

# 表单数据
data = {
    #"client_id":"ohRz0P8Kc696Szc5WQ",
    #"client_secret":"FJbG)i5T1L+n3w(Xrpw%2la^#l1$(vI8",
    "code": "nePVYO",
    "grant_type": "authorization_code",
    "scope": "tasks:read tasks:write",
    "redirect_uri": "https://baidu.com/"
}

# 设置 Content-Type
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

print("\nToken Request:")
print("URL:", url)
print("Headers:", headers)
print("Data:", data)

response = requests.post(
    url,
    headers=headers,
    data=data,
    auth=auth
)

print("\nToken Response:")
print("Status:", response.status_code)
print("Body:", response.text)


Token Request:
URL: https://dida365.com/oauth/token
Headers: {'Content-Type': 'application/x-www-form-urlencoded'}
Data: {'code': 'nePVYO', 'grant_type': 'authorization_code', 'scope': 'tasks:read tasks:write', 'redirect_uri': 'https://baidu.com/'}

Token Response:
Status: 400
Body: 
